In [ ]:
import numpy as np
import plotly.graph_objects as go

# Parameters
dt = 100          # time step (s)
T = 3600 * 4      # total time (s)
N = int(T / dt)
lat = 45 * np.pi / 180
f = 2 * 7.2921e-5 * np.sin(lat)  # Coriolis parameter

# Initial conditions
u, v = 0, 10  # m/s
x, y = [0], [0]

for _ in range(N):
    du = f * v * dt
    dv = -f * u * dt
    u += du
    v += dv
    x.append(x[-1] + u * dt)
    y.append(y[-1] + v * dt)

# Plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='red')))
fig.update_layout(title='2D Coriolis Effect – Northern Hemisphere',
                  xaxis_title='X Position (m)',
                  yaxis_title='Y Position (m)',
                  xaxis=dict(scaleanchor="y", scaleratio=1),
                  showlegend=False)
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Constants
R = 6.371e6
Omega = 7.2921e-5
dt = 60
T = 20 * 3600
N = int(T / dt)

# Initial position (equator)
theta = np.pi / 2
phi = 0
v = 200  # m/s northward

theta_list = [theta]
phi_list = [phi]

# Initial velocities
v_theta = -v / R
v_phi = 0

for _ in range(N):
    f = 2 * Omega * np.cos(theta)
    v_phi += (f * v * dt) / R
    theta += v_theta * dt
    phi += v_phi * dt
    theta_list.append(theta)
    phi_list.append(phi)

# Cartesian coordinates
x = R * np.sin(theta_list) * np.cos(phi_list)
y = R * np.sin(theta_list) * np.sin(phi_list)
z = R * np.cos(theta_list)

# Sphere (Earth) mesh
u, v = np.linspace(0, 2 * np.pi, 60), np.linspace(0, np.pi, 30)
u, v = np.meshgrid(u, v)
x_sphere = R * np.sin(v) * np.cos(u)
y_sphere = R * np.sin(v) * np.sin(u)
z_sphere = R * np.cos(v)

# Plotly figure
fig = go.Figure()

# Earth surface
fig.add_trace(go.Surface(x=x_sphere, y=y_sphere, z=z_sphere,
                         colorscale='Blues', showscale=False, opacity=0.5))

# Trajectory
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines',
                           line=dict(color='red', width=6),
                           name='Trajectory'))

fig.update_layout(scene=dict(aspectmode='data'),
                  title='Coriolis Effect on a Rotating Sphere',
                  showlegend=False)

fig.show()


In [ ]:
import plotly.figure_factory as ff
import numpy as np

# Grid setup
x, y = np.meshgrid(np.linspace(-1, 1, 20), np.linspace(-1, 1, 20))
p = -x  # linear pressure field (decreasing in x)

# Pressure gradient
dpdx = np.gradient(p, axis=1)
dpdy = np.gradient(p, axis=0)

# Constants
rho = 1.225
f = 1e-4

# Geostrophic wind: perpendicular to gradient
u = -1 / (rho * f) * dpdy
v =  1 / (rho * f) * dpdx

# Normalize for quiver plotting
mag_p = np.sqrt(dpdx**2 + dpdy**2)
mag_g = np.sqrt(u**2 + v**2)

# Avoid division by zero
mag_p[mag_p == 0] = 1
mag_g[mag_g == 0] = 1

# Normalized vectors (unit length arrows)
dpdx_n = dpdx / mag_p
dpdy_n = dpdy / mag_p
u_n = u / mag_g
v_n = v / mag_g

# Create two quiver plots: pressure gradient (blue), geostrophic wind (red)
fig = ff.create_quiver(x, y, dpdx_n, dpdy_n, scale=0.1,
                       arrow_scale=0.2, line=dict(color='blue'),
                       name='Pressure Gradient')
fig2 = ff.create_quiver(x, y, u_n, v_n, scale=0.1,
                        arrow_scale=0.2, line=dict(color='red'),
                        name='Geostrophic Wind')

# Merge the traces
fig.add_traces(fig2.data)
fig.update_layout(title="Geostrophic Balance: Red = Wind, Blue = Pressure Gradient", height=600,)
fig.show()


In [ ]:
import plotly.figure_factory as ff
import numpy as np

# Grid
x, y = np.meshgrid(np.linspace(-1, 1, 20), np.linspace(-1, 1, 20))
p = -x  # pressure decreases left to right

# Pressure gradient (pointing from high to low)
dpdx = np.gradient(p, axis=1)
dpdy = np.gradient(p, axis=0)

# Wind now moves directly down gradient (like gravity pulling it)
u = -dpdx
v = -dpdy

# Normalize for plotting
mag = np.sqrt(u**2 + v**2)
mag[mag == 0] = 1  # avoid division by zero
u_n = u / mag
v_n = v / mag

# Plot
fig = ff.create_quiver(x, y, u_n, v_n, scale=0.2,
                       arrow_scale=0.4, line=dict(color='green'),
                       name='Wind (No Coriolis)')
fig.update_layout(title="Wind Without Coriolis Force (Follows Pressure Gradient)")
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Constants
R = 6.371e6
Omega = 7.2921e-5
dt = 60
T = 20 * 3600
N = int(T / dt)

# Start near equator
theta = np.pi / 2  # colatitude
phi = 0
v_r = 200  # m/s northward

# Initialize arrays
theta_list = [theta]
phi_list = [phi]

# Velocity components (theta = north-south, phi = east-west)
v_theta = -v_r / R
v_phi = 0

for _ in range(N):
    # Local Coriolis effect
    f = 2 * Omega * np.cos(theta)
    v_phi += f * v_r * dt / R  # apply Coriolis deflection
    
    theta += v_theta * dt
    phi += v_phi * dt

    theta_list.append(theta)
    phi_list.append(phi)

# Convert to Cartesian coordinates
x = R * np.sin(theta_list) * np.cos(phi_list)
y = R * np.sin(theta_list) * np.sin(phi_list)
z = R * np.cos(theta_list)

# Earth mesh
u, v = np.linspace(0, 2*np.pi, 60), np.linspace(0, np.pi, 30)
u, v = np.meshgrid(u, v)
x_s = R * np.sin(v) * np.cos(u)
y_s = R * np.sin(v) * np.sin(u)
z_s = R * np.cos(v)

# Plot
fig = go.Figure()

# Earth surface
fig.add_trace(go.Surface(x=x_s, y=y_s, z=z_s,
                         colorscale='Blues', showscale=False, opacity=0.5))

# Trajectory with Coriolis
fig.add_trace(go.Scatter3d(x=x, y=y, z=z,
                           mode='lines',
                           line=dict(color='red', width=5),
                           name='With Coriolis'))

fig.update_layout(scene=dict(aspectmode='data'),
                  title='Air Parcel Motion With Coriolis (Curving Eastward)')
fig.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Grid
x, y = np.meshgrid(np.linspace(-2, 2, 30), np.linspace(-2, 2, 30))

# Wind field: circular vortex (non-divergent)
u = -y
v = x

# Compute vorticity (∂v/∂x - ∂u/∂y)
dvdx = np.gradient(v, axis=1) / (x[0, 1] - x[0, 0])
dudy = np.gradient(u, axis=0) / (y[1, 0] - y[0, 0])
zeta = dvdx - dudy

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
strm = ax.streamplot(x, y, u, v, color='k', density=1.2)
c = ax.contourf(x, y, zeta, cmap='RdBu', levels=20)
plt.colorbar(c, ax=ax, label='Relative Vorticity')
ax.set_title('Relative Vorticity Field (2D)')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_aspect('equal')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Grid (let y-axis represent latitude from -30° to 30°)
x, y = np.meshgrid(np.linspace(-2, 2, 30), np.linspace(-30, 30, 30))  # y in degrees
phi = np.deg2rad(y)  # latitude in radians

# Wind field: circular pattern
u = -y / 30  # normalize so units match approx
v = x / 2

# Relative vorticity (∂v/∂x - ∂u/∂y)
dvdx = np.gradient(v, axis=1) / (x[0, 1] - x[0, 0])
dudy = np.gradient(u, axis=0) / (y[1, 0] - y[0, 0])
zeta = dvdx - dudy

# Planetary vorticity f
Omega = 7.2921e-5  # rad/s
f = 2 * Omega * np.sin(phi)

# Absolute vorticity
zeta_abs = zeta + f

# Plot
fig, ax = plt.subplots(figsize=(16, 6))
strm = ax.streamplot(x, y, u, v, color='k', density=1.2)
c = ax.contourf(x, y, zeta_abs, cmap='RdBu', levels=20)
plt.colorbar(c, ax=ax, label='Absolute Vorticity')
ax.set_title('Absolute Vorticity (Relative + Planetary)')
ax.set_xlabel('x')
ax.set_ylabel('Latitude (°)')
# ax.set_aspect('equal')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Setup
lat = np.linspace(-30, 60, 500)  # degrees
phi = np.deg2rad(lat)            # radians
Omega = 7.2921e-5                # Earth's rotation rate
f = 2 * Omega * np.sin(phi)      # Coriolis parameter

# Assume absolute vorticity is conserved
zeta_abs_initial = f[0] + 5e-5  # initial total vorticity (arbitrary)

# Compute relative vorticity needed to conserve ζ + f
zeta = zeta_abs_initial - f

# Plot
plt.figure(figsize=(8, 5))
plt.plot(lat, f, label='Planetary Vorticity f(φ)', color='blue')
plt.plot(lat, zeta, label='Relative Vorticity ζ(φ)', color='red')
plt.plot(lat, zeta + f, '--', label='Absolute Vorticity ζ + f', color='black')

plt.title('Conservation of Absolute Vorticity Along Flow Line')
plt.xlabel('Latitude (°)')
plt.ylabel('Vorticity (1/s)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
Omega = 7.2921e-5  # Earth's rotation rate (rad/s)
a = 6.371e6        # Earth radius (m)

# Latitude range around a reference latitude (e.g., 45°)
phi0_deg = 45
phi0 = np.deg2rad(phi0_deg)

# y = distance north/south from φ₀ (meters)
dy = np.linspace(-2e6, 2e6, 500)  # +/- 2000 km
phi = phi0 + dy / a              # convert to latitude in radians

# True Coriolis parameter
f_true = 2 * Omega * np.sin(phi)

# Beta-plane approximation
f0 = 2 * Omega * np.sin(phi0)
beta = 2 * Omega * np.cos(phi0) / a
f_beta = f0 + beta * dy

# Plot
plt.figure(figsize=(8, 5))
plt.plot(dy / 1000, f_true, label='True $f(\\phi)$', linewidth=2)
plt.plot(dy / 1000, f_beta, '--', label='Beta-Plane Approximation', linewidth=2)
plt.axvline(0, color='gray', linestyle=':', label=f'Latitude = {phi0_deg}°')
plt.xlabel('Distance from Reference Latitude (km)')
plt.ylabel('Coriolis Parameter $f$ (1/s)')
plt.title('Beta-Plane Approximation vs True Coriolis Parameter')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
u = 10  # m/s background flow
beta = 2e-11  # 1/m/s
l = 0  # purely zonal wave (no meridional structure)

# Wavenumbers
k = np.linspace(1e-7, 5e-6, 500)  # 1/m
omega = u * k - beta * k / (k**2 + l**2)
c = omega / k  # phase speed

# Plot
plt.figure(figsize=(8, 5))
plt.plot(1/k / 1000, c, label='Phase Speed (c = ω/k)')
plt.axhline(u, color='gray', linestyle='--', label='Background Flow (u)')
plt.axhline(0, color='black', linestyle=':')
plt.xlabel('Wavelength (km)')
plt.ylabel('Phase Speed (m/s)')
plt.title('Rossby Wave Phase Speed vs Wavelength')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Wave parameters
u = 10         # m/s (mean flow)
beta = 2e-11   # 1/m/s
k = 2 * np.pi / (3e6)  # zonal wavenumber (2π / 3000 km)
l = 0          # purely zonal
A = 1          # amplitude

# Dispersion relation
omega = u * k - beta * k / (k**2 + l**2)

# Spatial and temporal domain
x = np.linspace(0, 6e6, 500)  # 0 to 6000 km
t = np.linspace(0, 2*24*3600, 200)  # 2 days in seconds

# Create figure
fig, ax = plt.subplots(figsize=(8, 4))
line, = ax.plot(x / 1000, A * np.cos(k * x), color='royalblue')
ax.set_ylim(-1.5, 1.5)
ax.set_xlim(0, 6000)
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Streamfunction ψ')
ax.set_title('Rossby Wave Animation (1D)')

# Animation function
def update(frame):
    psi = A * np.cos(k * x - omega * t[frame])
    line.set_ydata(psi)
    return line,

ani = animation.FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)

plt.show()
